In [10]:
import xml.etree.ElementTree as ET
import pandas as pd
import numpy as np
import os

In [34]:
def parse(xml_file):
    tree = ET.parse(xml_file)  
    root = tree.getroot()

    rows = []
    for topic in root.findall('topic'):
        topic_id = topic.find('id').text if topic.find('id') is not None else ''
        for result in topic.find('results').findall('result'):
            result_id = result.find('id').text if result.find('id') is not None else ''
            rel = result.find('rel').text if result.find('rel') is not None else ''
            text = result.find('text').text if result.find('text') is not None else ''
            rows.append({
                'topic_id': topic_id,
                'result_id': result_id,
                'rel': rel,
                'text': text
            })

    df = pd.DataFrame(rows)
    df.to_csv('output.csv', mode='a', index=False, header=not os.path.exists('output.csv'))

parse("combined.xml")

In [35]:
df = pd.read_csv('output.csv')
def extract_topic_name(url):
    last_part = url.split('chronicling-america-')[-1] 
    topic = last_part.replace('-', ' ')
    return topic.title()

df['topic_id'] = df['topic_id'].apply(extract_topic_name)
print(np.unique(df['topic_id']))
df

['1904 Presidential Election' 'Babe Ruth' 'Cassius Marcellus Clay Jr'
 'Clara H Barton' 'Early Women Aviation/Selected Articles' 'Ellis Island'
 'Harry Houdini' 'Indianapolis 500' 'Lindbergh Baby Kidnapping'
 'Marie Curie' 'Prohibition']


,topic_id,result_id,rel,text
0,Indianapolis 500,\n https://www.loc.gov/resource/sn830...,1.0,DEATH THRILLS CROWD\n WATCHING MOTOR ...
1,Indianapolis 500,\n https://www.loc.gov/resource/sn830...,1.0,The eyes of the motoring public\n wer...
2,Indianapolis 500,\n https://www.loc.gov/resource/sn920...,1.0,WORLD'S GREATEST AUTO RACE\n TO OCCUR...
3,Indianapolis 500,\n https://www.loc.gov/resource/sn850...,1.0,"Race For $50,000 at Indianapolis Has\n ..."
4,Indianapolis 500,\n https://www.loc.gov/resource/sn850...,1.0,Death Merely Spectator As\n National ...
...,...,...,...,...
158,Prohibition,https://www.loc.gov/item/10000343/,1.0,PROHIBITORY LEGISLATION. \n\nLocal Option 100 ...
159,Prohibition,https://www.loc.gov/item/09034898/,1.0,PROHIBITION CAMPAIGN DOCUMENTS. \n\n\nT HE NAT...
160,Prohibition,https://www.loc.gov/item/13019196/,1.0,"ADDRESS OF GOVERNOR W. R. STUBBS OF KANSAS, \n..."
161,Prohibition,https://www.loc.gov/item/23013066/,1.0,\nPROHIBITION AND ITS ENFORCEMENT \n\n\nPAGE \...
